In [72]:
!pip install numpy
!pip install opencv-python
!pip install tkcalendar
!pip install pillow
!pip install protobuf==3.20.*
!pip install tensorflow==2.3.0 keras==2.4.0

     ---------------------------------------- 0.0/40.9 kB ? eta -:--:--
     ---------------------------------------- 0.0/40.9 kB ? eta -:--:--
     ---------------------------------------- 0.0/40.9 kB ? eta -:--:--
     ---------------------------------------- 0.0/40.9 kB ? eta -:--:--
     ---------- ----------------------------- 10.2/40.9 kB ? eta -:--:--
     ------------------- ------------------ 20.5/40.9 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 40.9/40.9 kB 393.2 kB/s eta 0:00:00


In [6]:
import cv2
import numpy as np
import requests
import tkinter
from datetime import datetime
from tkinter import *
from tkinter import ttk
from tkinter import filedialog, PhotoImage
from tkcalendar import DateEntry #日期選單
from tkinter import Text, Scrollbar #捲動式
from tkinter import messagebox #對話框
from PIL import Image, ImageTk, ImageDraw, ImageFont

from keras.models import load_model

"""
1. 學生清單一覽表
"""
student_list = [
    {'name': 'nobody', 'id': 0},
    {'name': 'meteor', 'id': 1},
    {'name': 'VE', 'id': 2},
    {'name': 'Naninu', 'id': 3},
    {'name': 'Meng', 'id': 4},
    {'name': 'Kuan', 'id': 5},
    {'name': 'Hajimi', 'id': 6},
    {'name': 'Edd', 'id': 7},
    {'name': 'dog123', 'id': 8},
    {'name': 'Bear', 'id': 9},
    {'name': 'Teacher', 'id': 10}
]


"""
2. AI 圖像辨識模型
"""
# Load the model
model = load_model("./models/keras_model.h5", compile=False)
labels = open("./models/labels.txt", "r", encoding="UTF-8").readlines()


"""
3. Python GUI 設計
"""
root = Tk()
root.geometry("800x600")
root.iconbitmap('./images/icon.ico')

#設定視窗物件的標題
root.title("AI人臉辨識打卡系統")
root.configure(bg='#fffacd') #背景色


title = Label(root, height=2, width=47,
              fg='#ffffe0', bg='#A98B73',
              text="人臉辨識自動打卡系統",
              font=('微軟正黑體 21 bold')).pack()

label = Label(root)

location_x = 20
location_y = 65

"""
3.1 Label： Logo
"""
# Create a photoimage object of the image in the path
logo = Image.open("./images/Logo.png")

final_logo = ImageTk.PhotoImage(logo)

lblLogo = Label(image=final_logo,bg='#fffacd')
lblLogo.image = final_logo

# Position image
lblLogo.place(x=10+location_x, y=13+location_y)
    
"""
3.2 Button 1: 開啟攝影鏡頭
3.2.1 Webcam 鏡頭與顯像 size 設定
"""
#儲存開始簽到、結束簽到時間
checkin_or_not = False  # 初始化簽到狀態為F
start_time = None  # 初始化開始點名時間為空

def to_pil(img, label, x, y, w, h):
    img = cv2.resize(img, (w, h))
    image = Image.fromarray(img)
    pic = ImageTk.PhotoImage(image)
    label.configure(image=pic)
    label.image = pic
    label.place(x=x, y=y)

def choose(): #開始點名初始化
    global cap, image, title
    global checkin_or_not,start_time,end_time
    cap = cv2.VideoCapture(0)
    
    if checkin_or_not == False and start_time == None:
        course_name = entry_course.get() #儲存課程名稱
        if course_name == "":
            messagebox.showwarning("警告：尚未輸入課程名稱","尚未輸入課程名稱，請先輸入課程名稱後再開始點名！")
        else:
            lbxShoppingCart.delete(0, 'end') #清空前次點名紀錄
            start_time = (datetime.now()).strftime("%Y-%m-%d %H:%M:%S")
            checkin_or_not = True
            show()
    
    
def cv2ImgAddText(img, text, left, top, textColor=(0,255,0), textSize=20):   #將WEBCAM的FRAME加上文字
    if (isinstance(img, np.ndarray)):  #判斷是否為 OpenCV 圖片類型
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    draw = ImageDraw.Draw(img)
    font_path = "./fonts/NotoSansTC-Regular.otf"  #設定字型路徑
    fontText = ImageFont.truetype(font_path, textSize, encoding="utf-8")
    draw.text((left, top), text, textColor, font=fontText)
    return cv2.cvtColor(np.asarray(img), cv2.COLOR_RGB2BGR)

"""
3.2.2 AI 影像識別計算函數
"""    
def recognition(item_image):
    # Resize the raw image into (224-height,224-width) pixels.
    imageAI = cv2.resize(item_image, (224, 224), interpolation=cv2.INTER_AREA)
    # Make the image a numpy array and reshape it to the models input shape.
    imageAI = np.asarray(imageAI, dtype=np.float32).reshape(1, 224, 224, 3)
    # Normalize the image array
    imageAI = (imageAI / 127.5) - 1
        
    # Have the model predict what the current image is. Model.predict
    # returns an array of percentages. Example:[0.2,0.8] meaning its 20% sure
    # it is the first label and 80% sure its the second label.
    probabilities = model.predict(imageAI)
    # Print what the highest value probabilitie label
    #image_class = labels[np.argmax(probabilities)]
    image_class = np.argmax(probabilities)    
    return image_class    

def show():
    global title
    scaling_factor = 0.5
    ret, frame = cap.read()
    
    if frame is not None:
        frame = cv2.resize(frame, None,
                           fx=scaling_factor,
                           fy=scaling_factor,
                           interpolation=cv2.INTER_AREA)

        pid = recognition(frame)
        title = student_list[pid].get("name")

        # 在這裡將名字顯示在影像上
        frame = cv2.flip(frame, 1)
        frame = cv2ImgAddText(frame, title, 10, 10, (255, 0, 0), 25) #字體大小顏色

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        to_pil(rgb, label, 30 + location_x, 150 + location_y, 230, 280)

        label.after(30, show)  
    
#開始點名btn
btnSwitch = Button(root,
                   text="開始點名",
                   font=("微軟正黑體 12"),
                   command=choose,bg="#deb887", width=15, height=2)
btnSwitch.place(x=210+location_x, y=20+location_y)    

#輸入課程名稱
lblCourseName = Label(root, height=2,
                      fg='brown',bg='#fffacd',
                      text="課程名稱：",
                      font=('微軟正黑體 18 bold'))
lblCourseName.pack()
lblCourseName.place(x=210+location_x, y=80+location_y)
#lblCourseName.config(text="課程名稱：")

    
#課程名稱輸入框
entry_course = Entry(root, font=('微軟正黑體',15), width=20) 
entry_course.pack(pady=10)
entry_course.place(x=330+location_x, y=100+location_y)


"""
3.3 Button 2: 關閉攝影鏡頭
"""
def close():
    cap.release()
    # Close all active windows
    cv2.destroyAllWindows()

def on_closing():
    close()  # 當視窗關閉時一併關閉攝影鏡頭
    root.destroy()
    return
    
"""
3.4 Button 3: 簽到後顯示於記錄欄，以及從記錄欄移除紀錄
"""
def hide_message():
    chekin_ms.config(text="")  # 清空文字
    
def show_message(feedback):
    chekin_ms.config(text=feedback)    
    chekin_ms.place(x=130+location_x, y=450+location_y)
    chekin_ms.after(5000, hide_message)  # 5000毫秒後隱藏  

def buy(): #寫if資料庫收到點名成功再呼叫show_message()顯示簽到完成
    global title #title是目前辨識到的名字 Ex:edd
    if title == 'nobody':
        message = '未偵測到臉孔'
    else:
        url = "https://u110029024.000webhostapp.com/AI_ML/update.php" # Define the URL of your PHP script
        current_time = datetime.now() # 獲取系統時間
        # 將時間格式化為指定的字串格式
        formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")# 將時間格式化為指定的字串格式EX:2023-10-12 17:47:27
        # Define the data to be sent in the POST request
        data = {
            'name': title , 
            'time':formatted_time
        }

        # Send the POST request with data
        response = requests.post(url, data=data)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            if response.text == 'dbpass':
                message = '簽到成功'
                lbxShoppingCart.insert('end', f"{title} {formatted_time}")  #從最後一個位置開始加入值
            else:
                message = '簽到失敗(db)'
        else:
            message = '簽到失敗(php)'
    show_message(message)

def delete():  #刪除點選位置的簽到記錄
    if(lbxShoppingCart.curselection() != ()):
        selected_value = lbxShoppingCart.get(lbxShoppingCart.curselection()[0]) #selected_value格式為edd 2023-10-29
        # 以空格分割字符串
        parts = selected_value.split(' ')
        time = parts[1]+' '+parts[2]
        url = "https://u110029024.000webhostapp.com/AI_ML/delete.php" # Define the URL of your PHP script
        # Define the data to be sent in the POST request
        data = {
            'name': parts[0] , 
            'time': time
        }

        # Send the POST request with data
        response = requests.post(url, data=data)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            if response.text == 'pass':
                message = '刪除成功'
                lbxShoppingCart.delete(lbxShoppingCart.curselection()) #刪除目前所選取的欄位
            else:
                message = '刪除失敗(db)'
        else:
            message = '刪除失敗(php)'
        show_message(message)
    
btnBuy = Button(root,
                text="簽到",
                font=("微軟正黑體 12"),
                command=buy,bg="#deb887",width=8, height=2)
btnBuy.place(x=30+location_x, y=450+location_y)

chekin_ms = Label(root, text="", fg='red',bg='#fffacd', font=("Arial 14 bold"), height=2)
chekin_ms.place(x=150+location_x, y=450+location_y)

btnDelete = Button(root,
                text="移除",
                font=("微軟正黑體 12"),
                command=delete,bg="#deb887", width=9, height=2)
btnDelete.place(x=630+location_x, y=90+location_y)



"""
3.6 Button 5: 查詢點名紀錄 
"""
#點選查詢跳出新視窗 search_window

result_text = None  # 將 result_text 定義為全局變數
date_var = StringVar()  # date_var儲存被選擇的日期
course_names=[]        

def search_window():
    #日期選擇完成即更新課程選擇欄
    def date_change(*args):
        global course_names, course_var
        selected_date = date_var.get()
        url = "https://u110029024.000webhostapp.com/AI_ML/list_query.php"
        data = {'date': selected_date}
        response = requests.post(url, data=data)
        if response.status_code == 200:
            course_names = response.text.split(',')
            course_combobox['values'] = course_names
            #course_var.set(course_names[0])
            
    global result_text  # 告訴 Python 使用全局變數 result_text
    
    
    search_window = Toplevel(root)  # root是主窗口
    search_window.configure(bg='#fffacd')
    search_window.iconbitmap('./images/icon2.ico')
    search_window.title("查詢點名紀錄")
    search_window.geometry("500x500")  # 設定新視窗的大小
    label = Label(search_window, text="選擇欲查詢日期、課程及學生名稱：",bg='#fffacd', font=("微軟正黑體 12 bold"), height=2)
    label.pack()

    # 日期選擇框
    #date_var = StringVar()  # date_var儲存被選擇的日期
    date_entry = DateEntry(search_window, width=20, font=("Arial", 12), background='#cd853f',
                           foreground='white', borderwidth=2, textvariable=date_var,
                           date_pattern='yyyy-mm-dd')
    date_entry.pack(pady=10)
    date_entry.place(x=100, y=50)
    
    date_var.trace_add('write', date_change)
    
    # 下拉式清單-課程清單
    course_var = StringVar()
    #course_var.set(course_names[0]) #預設為第一筆(bug未顯示)
    course_combobox = ttk.Combobox(search_window, textvariable=course_var, values=course_names,
                                    width=20, font=("Arial", 12))
    course_combobox.pack(pady=10)
    course_combobox.place(x=100, y=80)

    # 下拉式清單-學生清單
    #url = "https://u110029024.000webhostapp.com/AI_ML/list_query.php"
    #data = {'command': url}
    #response = requests.post(url, data=data)
    #temp_text = response.text
    #student_list = temp_text.split(',')
    student_names = ['all'] + [student['name'] for student in student_list] #清單值為資料庫中已打卡過的學生
    student_var = StringVar()
    student_var.set(student_names[0]) #預設為all
    student_combobox = ttk.Combobox(search_window, textvariable=student_var, values=student_names,
                                    width=20, font=("Arial", 12))
    student_combobox.pack(pady=10)
    student_combobox.place(x=100, y=110)
    
    
    # 查詢按鈕
    query_button = Button(search_window, text="查詢", font=("微軟正黑體 12"),
                          command=lambda: query(date_var, course_var,student_var),
                          bg="#deb887",width=10, height=2)
    query_button.pack(pady=10)
    query_button.place(x=330, y=63)
    
    # 定義 Scrollbar
    scroll_y = Scrollbar(search_window, orient="vertical")

    # 顯示結果的標籤 改為TEXT搭配捲動式
    result_text = Text(search_window, width=50, height=18,bg='#fafad2', 
                       yscrollcommand=scroll_y.set, font=("Arial", 12), wrap="word")
    result_text.pack(pady=20)
    result_text.place(x=20, y=150)
    
    # 設定 Scrollbar 與 result_text 之間的連結
    scroll_y.config(command=result_text.yview)
        
    
# 逐行插入查詢結果
def display_responses(response_list):
    # 清空之前的結果
    result_text.delete(1.0, END)

    # 逐行插入字串
    for response in response_list:
        result_text.insert(END, f"{response}\n")
        
def query(date_var,course_var,student_var):
    result_text.delete(1.0, "end")  # 清空之前的結果
    selected_date = date_var.get()
    selected_course=course_var.get()
    selected_student = student_var.get()
    temp_list = selected_course.split(' ')
    temp_list = temp_list[1].split('～')
    starttime = selected_date+' '+temp_list[0]
    endtime = selected_date+' '+temp_list[1]
    url = "https://u110029024.000webhostapp.com/AI_ML/query.php" # Define the URL of your PHP script
    # Define the data to be sent in the POST request
    data = {
        'name': selected_student , 
        'start':starttime,
        'end':endtime
    }
    
    # Send the POST request with data
    response = requests.post(url, data=data)
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        if response.text == 'norecord':
            response_list = ['查無點名紀錄！']
        else:
            #text = response.text
            response_list = response.text.split(',')  #ex:list[0]的內容應為：edd已於2023-10-30 17:39:01簽到成功！
    else:
        result_text.insert("end", "查詢失敗")
    
    #result_text.config(text=f"查詢日期：{selected_date}\n學生名字：{selected_student}")
    
    display_responses(response_list)
    #result_text.insert("end", f"查詢日期：{selected_date}\n學生名稱：{selected_student}\n")
    #result_text.insert("end", "0\n1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n12\n13\n14\n15\n16\n17\n") #測試用
    
#主視窗查詢按鈕    
btnCheckout = Button(root,
                text="查詢點名紀錄",
                font=("微軟正黑體 12"),
                command=search_window, bg="#deb887",width=15 , height=2)
btnCheckout.place(x=570+location_x, y=20+location_y)

"""
3.7 Listbox: 記錄欄
"""
lbxShoppingCart = Listbox(root, font=('Arial 16'),
                          width=36, height=14,
                          background='#ffffe6')
lbxShoppingCart.place(x=340, y=150+location_y)


"""
3.8 Label：彈跳視窗顯示未到人數與缺席學生
"""
# 3.2初始化關閉按鈕狀態為未按下、初始化開始點名時間為空

def close_checkin():
    global checkin_or_not, start_time, entry_course
    TotalAbsentee=None #儲存缺席人數

    
    if checkin_or_not == True:
        #lblCourseName.place(x=330+location_x, y=80+location_y)
        #lblCourseName.place(x=210+location_x, y=80+location_y)
        #lblCourseName.pack()
        url = 'https://u110029024.000webhostapp.com/AI_ML/total_absentee.php'
        # Define the data to be sent in the POST request
        data = {
            'start': start_time,
            'class_name':entry_course.get()
        }
        # Send the POST request with data
        response = requests.post(url, data=data)
        # Check if the request was successful (status code 200)
        if response.status_code == 200: #再傳absenteetotal
            #arrive_student = response.text.split(',')
            #show_message(response.text)
            temp_student = response.text.split(';')
            endtime = temp_student[1]
            arrive_student = temp_student[0].split(',') 
            #show_message(arrive_student)
            
            nonarrived_student = []
            #count = 0
            for student in student_list:
                #count+=1
                if student['name'] not in arrive_student:
                    nonarrived_student.append(student['name'])
            nonarrived_student.remove('nobody')

            #nonarrived_student #未到學生的list
            TotalAbsentee = len(nonarrived_student) 
            #lblCoursessName.config(text=f"未到人數：{TotalAbsentee}人")
            result=messagebox.showinfo(f"{endtime}結束點名",
                                       f"未到人數：{TotalAbsentee}人；未到學生名單：{nonarrived_student}")
            if result=="OK":
                entry_course.delete(0, 'end')
                #lblCourseName.pack()
                #lblCourseName.place(x=330+location_x, y=80+location_y)
                checkin_or_not = False
        else:
            messagebox.showwarning=("未到資訊抓取錯誤","未到資訊抓取錯誤")
            #lblCourseName.config(text=f"未到人數：抓取錯誤")
        #------需在這行之前將start_time回傳給資料庫，並判斷結束時間，由此段時間內抓取缺席人數TotalAbsentee
        start_time = None
        checkin_or_not = False        
    
    #lblCourseName.place(x=330+location_x, y=80+location_y)
    close()
    
#結束點名btn        
btnExit = Button(root,
                 text="結束點名",
                 font=("微軟正黑體 12"),
                 command=close_checkin,bg="#deb887", width=15, height=2)
btnExit.place(x=390+location_x, y=20+location_y)  #x=390+location_x, y=15+location_y   

var = StringVar()


#lbxShoppingCart.insert(END,f"{##結束時間##}結束點名—未到學生：{nonarrived_student}")  #從最後一個位置加入缺席學生名字
#lbxShoppingCart.itemconfig(END, {'fg': 'red'}

# 設定關閉事件處理器（視窗關閉）
root.protocol("WM_DELETE_WINDOW", on_closing)
root.mainloop()